In [18]:
#숫자 데이터가 아닌 범주형 데이터가 있는 경우가 있다 
#범주형 데이터를 처리하는 모델 알아보자 

import pandas as pd 
from sklearn.model_selection import train_test_split

data = pd.read_csv('../../data/melb_data.csv')

# 독립변수 / 종속변수 분리 
y = data.Price  #종속변수
x = data.drop(['Price'], axis=1)

#훈련용 / 검증용 데이터 분리
x_train, x_test, y_train, y_test =\
train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=0)

# 결측치가 있는 열 삭제
cols_with_missing = [col for col in x_train.columns if x_train[col].isnull().any()] 
x_train.drop(cols_with_missing, axis=1, inplace=True)
x_test.drop(cols_with_missing, axis=1, inplace=True)

# 카디널리티는 열의 고유 값 수를 의미 
#상대적으로 카디널리티가 낮은 범주형 열을 선택함(임의적)

#훈련데이터의 열의 고윳값이 10보다 작고 데이터 타입이 object인 열 >> low_cardinality_cols
low_cardinality_cols = [cname for cname in x_train.columns if x_train[cname].nunique() < 10
                        and x_train[cname].dtype == 'object']

#숫자형 데이터
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

# 선택한 열만 유지 
my_cols = low_cardinality_cols + numerical_cols
x_train = x_train[my_cols].copy()
x_test = x_test[my_cols].copy()


In [19]:
x_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [20]:
s = (x_train.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [21]:
#랜덤포레스트 모델 함수 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(x_train, x_test, y_train, y_test):
    #모델 구축
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return mean_absolute_error(y_test, y_pred)
    

In [22]:
# 접근법 1 (범주형 변수 삭제)
drop_x_train = x_train.select_dtypes(exclude=['object'])
drop_x_test = x_test.select_dtypes(exclude=['object'])

print("접근법 1의 MAE(범주형 변수 삭제): ", 
      score_dataset(drop_x_train, drop_x_test, y_train, y_test))


접근법 1의 MAE(범주형 변수 삭제):  183550.22137772635


In [23]:
# 접근법 2 >>  ordinal encoding (서수 인코딩)
# 순서형 범주형 데이터일 경우 적용

from sklearn.preprocessing import OrdinalEncoder

label_x_train = x_train.copy()
label_x_test = x_test.copy()

oe = OrdinalEncoder()
label_x_train[object_cols] = oe.fit_transform(x_train[object_cols])
label_x_test[object_cols] = oe.transform(x_test[object_cols])

print('접근법 2의 MAE(서수인코딩 ) :',score_dataset(label_x_train, label_x_test, y_train, y_test))


접근법 2의 MAE(서수인코딩 ) : 175062.2967599411


In [26]:
# 접근법 3 >> OneHot encoding(원핫인코딩)
# 0과 1로만 인코딩 

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_x_train = pd.DataFrame(ohe.fit_transform(x_train[object_cols]))
ohe_x_test = pd.DataFrame(ohe.transform(x_test[object_cols]))

#원핫인코딩 제거 인덱스 
ohe_x_train.index = x_train.index
ohe_x_test.index = x_test.index

#원핫 인코딩으로 대체됨 (범주형 데이터 열 제거)
num_x_train = x_train.drop(object_cols, axis=1)
num_x_test = x_test.drop(object_cols, axis=1)

#숫자 특성에 원-핫 인코딩된 열 추가
OH_x_train = pd.concat([num_x_train, ohe_x_train], axis=1)
OH_x_test = pd.concat([num_x_test, ohe_x_test], axis=1)

# 문자형인 데이터 열 확인 
OH_x_train.columns = OH_x_train.columns.astype(str)
OH_x_test.columns = OH_x_test.columns.astype(str)

print("접근법 3의 MAE(원핫인코딩 ) :", score_dataset(OH_x_train, OH_x_test, y_train, y_test))

접근법 3의 MAE(원핫인코딩 ) : 176703.63810751104


In [27]:
#3개의 접근법중 2,3이 성능이 좋음 
# 1번인 범주형 열 삭제 방법은 성능이 안좋지만 경우에 따라 다름